In [1]:
from keras.models import load_model
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input 

In [2]:
model = load_model('VGG16_MaskMonitor.h5')

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

https://medium.com/@nutanbhogendrasharma/deep-convolutional-networks-vgg16-for-image-recognition-in-keras-a4beb59f80a7


In [3]:


while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray )  

    for (x,y,w,h) in faces:
    
        #---------------------------obtain ROI in orignal image:
        face_img=img[y:y+w,x:x+w]
        
        #------------------------------------------------------------------ putting ROI for preprocessig:
        
        # load an image from file
        resized=cv2.resize(face_img,(224,224))
         
        # convert the image pixels to a numpy array
        image = img_to_array(resized)
        #reshape as the model expect a one or more images , sending a 4 dimensional array:
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
         
        # prepare the image for the VGG model(substracting mean RGB value from the training set with each pixel):
        proImage = preprocess_input(image)
        
        
        #-------------------------------------------------------------puting the preprocessed image into model.predict()
        result=model.predict(proImage)

        label=np.argmax(result,axis=1)[0]
      
         
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
       
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

In [ ]:

         '''
        #code to put frame detected without mask into our database :
        if(label == 1):
            #hence no mask
            img = img[y:y+w,x:x+w]
            cv2.imwrite('folToStore/default.png', img)'''